In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
parentdir = os.path.dirname(parentdir)
sys.path.insert(0,parentdir)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/My Drive/D2FM')
module_path = '/content/drive/My Drive/D2FM'
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
!pip install pykalman

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.9/228.9 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pykalman: filename=pykalman-0.9.5-py3-none-any.whl size=48442 sha256=511b82d0c12ddf725a9e6d486704f05f7495d8401894b954c40eaa7924952627
  Stored in directory: /root/.cache/pip/wheels/76/33/ef/5f332226e13a5089c6dd4b01cc2bcb59491d18f955fa2d3807
Successfully built pykalman


In [5]:
from models.ddfm import *

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from typing import Tuple
from tensorflow import keras
from tensorflow.keras import layers
from statsmodels.tsa.statespace.dynamic_factor_mq import DynamicFactorMQ as DFM
from sklearn.model_selection import ParameterGrid
from typing import Tuple
import numpy as np
from pykalman import KalmanFilter

In [7]:
#from google.colab import files
#uploaded = files.upload()

In [8]:
import warnings
warnings.filterwarnings("ignore")
import random
from sklearn.metrics import mean_squared_error as mse
from typing import Tuple
from sklearn.preprocessing import StandardScaler

In [9]:
df_train = pd.read_csv('/content/drive/MyDrive/D2FM/scaled_train.csv')
df_test=pd.read_csv('/content/drive/MyDrive/D2FM/scaled_test.csv')


In [10]:
df_train.columns

Index(['Index', 'RPI', 'W875RX1', 'DPCERA3M086SBEA', 'INDPRO', 'IPFPNSS',
       'IPFINAL', 'IPCONGD', 'IPDCONGD', 'IPNCONGD',
       ...
       'AAAFFM', 'BAAFFM', 'EXSZUSx', 'EXJPUSx', 'EXUSUKx', 'EXCAUSx',
       'DTCOLNVHFNM', 'DTCTHFNM', 'INVEST', 'FEDFUNDS'],
      dtype='object', length=119)

In [11]:
df_train_idx=df_train.set_index("Index")
df_test_idx=df_test.set_index("Index")

# Para 6 factores:

In [13]:
ddfm6 = DDFM(df_train_idx, structure_encoder=(288, 144, 96, 6), lags_input=1, factor_oder=2,
                             use_bias=True, link='tanh', max_iter=1000)
ddfm6.fit()

@Info - Note: Sorting data.
18/18 [==============================] - 0s 3ms/step
@Info: Convergence achieved in 5 iterations - new loss: 0.5898081148109916 - delta: 0.00023514336693640313 < 0.0005


In [12]:
last_state, last_state_var_cov_matrix = ddfm6.get_last_state()

In [13]:
last_state.shape

(130,)

In [15]:
steps_ahead = df_test_idx.shape[0]  # for example, predict 5 steps ahead
predictions6 = ddfm6.predict(last_state, last_state_var_cov_matrix, steps_ahead)
#predictions7['states'].shape
y_t_k_6=ddfm6.predict_observations(last_state, last_state_var_cov_matrix , steps_ahead)
#df_test_first_12_rows = df_test_idx.iloc[:12, :]
test_array = df_test_idx.to_numpy()
mse_row=np.mean((test_array - y_t_k_6)**2, axis=1)
mse_145=np.mean(mse_row)
mae_145 = np.mean(np.abs(test_array - y_t_k_6))
rmsfe_145 = np.sqrt(np.mean((test_array - y_t_k_6) ** 2))
print("Mean Squared Error:", mse_145)
print("Mean Absolute Error:", mae_145)
print("Root Mean Squared Forecast Error:", rmsfe_145)

Mean Squared Error: 0.9581370593879495
Mean Absolute Error: 0.6841562075959188
Root Mean Squared Forecast Error: 0.9788447575524678


In [ ]:
#n = ddfm.state_space.F.shape[0]
#x_hat_start = np.zeros((1, n))
#sigma_x_hat_start = np.eye(ddfm.state_space.F.shape[0])
#x_hat_start.shape

In [ ]:
#dir(ddfm)

### factores filtrados

In [16]:
ff6=ddfm6.factors_filtered
ff6.shape

(577, 6)

In [17]:
f_hat_ddfm_idx6 = pd.DataFrame(ff6, columns=["f1", "f2", "f3", "f4", "f5", "f6"])

In [18]:
f_hat_ddfm_idx6.head()

,f1,f2,f3,f4,f5,f6
0,-0.410142,0.112761,0.302629,0.558938,0.401059,0.000000
1,0.447758,0.163437,0.118480,0.040392,-0.823330,-0.811879
2,-0.313457,0.372762,-0.210943,-0.101538,-0.723166,-0.269557
3,0.110398,-0.100178,0.158665,0.015998,-0.588298,-0.076621
4,-0.261000,0.496313,-0.163399,-0.049169,0.119703,-0.017937


In [19]:
f_hat_ddfm_idx6.index = df_train_idx.index
f_hat_ddfm6=f_hat_ddfm_idx6.reset_index()

In [20]:
f_hat_ddfm6.head()

,Index,f1,f2,f3,f4,f5,f6
0,1960-01-01,-0.410142,0.112761,0.302629,0.558938,0.401059,0.000000
1,1960-02-01,0.447758,0.163437,0.118480,0.040392,-0.823330,-0.811879
2,1960-03-01,-0.313457,0.372762,-0.210943,-0.101538,-0.723166,-0.269557
3,1960-04-01,0.110398,-0.100178,0.158665,0.015998,-0.588298,-0.076621
4,1960-05-01,-0.261000,0.496313,-0.163399,-0.049169,0.119703,-0.017937


In [24]:
f_hat_ddfm_idx6.to_csv('/content/drive/MyDrive/D2FM/f_hat_DDFM_fact6.csv', index=False)

## Klieber

In [12]:
ddfmk = DDFM(df_train_idx, structure_encoder=(126, 86, 46, 6), link='relu', max_iter=1000, batch_size=24, batch_norm=False)
ddfmk.fit()

@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 2 iterations - new loss: 0.6280691799468571 - delta: 0.0004362362686674703 < 0.0005


In [13]:
last_state, last_state_var_cov_matrix = ddfmk.get_last_state()

In [23]:
predictionsk = ddfmk.predict(last_state, last_state_var_cov_matrix, steps_ahead)
#predictions7['states'].shape
y_t_k_k=ddfmk.predict_observations(last_state, last_state_var_cov_matrix , steps_ahead)
#df_test_first_12_rows = df_test_idx.iloc[:12, :]
mse_row=np.mean((test_array - y_t_k_k)**2, axis=1)
mse_145=np.mean(mse_row)
mae_145 = np.mean(np.abs(test_array - y_t_k_k))
rmsfe_145 = np.sqrt(np.mean((test_array - y_t_k_k) ** 2))
print("Mean Squared Error:", mse_145)
print("Mean Absolute Error:", mae_145)
print("Root Mean Squared Forecast Error:", rmsfe_145)

Mean Squared Error: 0.9636234953094749
Mean Absolute Error: 0.6886829249102709
Root Mean Squared Forecast Error: 0.9816432627535702
